In [1]:
import json
import query_elasticsearch as es

# This is the gold training data from pilot 3 or 4
evidence_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/evidence.json"
claim_persp_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/claim_perspective.json"
annotation_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/gold_annotation.json"

with open(evidence_path, 'r', encoding='utf-8') as fin:
    evidences = json.load(fin)

with open(claim_persp_path, 'r', encoding='utf-8') as fin:
    persps = json.load(fin)
    
with open(annotation_path, 'r', encoding='utf-8') as fin:
    annotations = json.load(fin)

print(len(evidences))
print(len(persps))
print(len(annotations))

8122
4645
4645


In [2]:
import pandas as pd

adf = pd.DataFrame(annotations)
adf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4645 entries, 0 to 4644
Data columns (total 2 columns):
evidence_id       4645 non-null int64
perspective_id    4645 non-null int64
dtypes: int64(2)
memory usage: 72.7 KB


In [ ]:
# For each perspective generate evidence candidates
results = []
for p in persps:
    # top 50 candidates
    _cands = es.get_top_evidences(p['title'])
    cands = [(eid, score) for e, eid, score in _cands]
    results.append({
        'persp_id': p['id'],
        'candidates': cands
    })

# save results
out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene/pp_2_ev.json"

with open(out_path, 'w') as fout:
    json.dump(results, fout)


In [17]:
# For each perspective, measure top k accuracy of evidence candidates

path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene/pp_2_ev.json"

with open(path) as fin:
    results = json.load(fin)

def get_gold_evidence(perspective_id, adf):
    q = adf.loc[adf.perspective_id == perspective_id]
    if len(q) > 0:
        return q.iloc[0].evidence_id
    else:
        return None
    
k = [1, 10, 25, 50]
correct = [0 for _k in k]

for res in results:
    gold = get_gold_evidence(res['persp_id'], adf)
    for idx, _k in enumerate(k):
        cands = [eid for eid, score in res['candidates']]
        
        if gold in cands[:_k]:
            correct[idx] += 1

total = len(results)


for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.36404736275565125
Top 10 accuracy: 0.7791173304628632
Top 25 accuracy: 0.8716899892357374
Top 50 accuracy: 0.9121636167922498


In [ ]:
# The other way around: for each evidence, generate perspective candidates
from elasticsearch.exceptions import RequestError

results = []
for e in evidences:
    # top 50 candidates
    try:
        _cands = es.get_top_perspectives(e['content'])
    except RequestError:
        _cands = []
    cands = [(pid, score) for p, pid, score in _cands]
    results.append({
        'evidence_id': e['id'],
        'candidates': cands
    })

# save results
out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene/ev_2_pp.json"

with open(out_path, 'w') as fout:
    json.dump(results, fout)


In [5]:
# For each evidence, measure top k accuracy of perspective candidates

path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene_persp_evidence/ev_2_pp.json"

with open(path) as fin:
    results = json.load(fin)
    
def get_gold_perspective(evidence_id, adf):
    q = adf.loc[adf.evidence_id == evidence_id]
    if len(q) > 0:
        return q.iloc[0].perspective_id
    else:
        return None
    
k = [1, 10, 25, 50]
correct = [0 for _k in k]
total = 0

for res in results:
    gold = get_gold_perspective(res['evidence_id'], adf)
    
    if gold == None:
        continue
        
    total += 1
    for idx, _k in enumerate(k):
        if len(res['candidates']) > 0:
            cands = [pid for pid, score in res['candidates']]
            if gold in cands[:_k]:
                correct[idx] += 1

for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.38256189451022604
Top 10 accuracy: 0.7089343379978471
Top 25 accuracy: 0.7885898815931108
Top 50 accuracy: 0.8320775026910656
